In [36]:
# Import the dataset
from datasets import load_dataset

ds = load_dataset("EduardoPacheco/FoodSeg103")

In [37]:
from PIL import Image

train, validation = ds["train"], ds["validation"]

print(train[0]['image'])
print("----------------- // -----------------")
print(train[0]['label'])

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x384 at 0x7AD3919CFE60>
----------------- // -----------------
<PIL.PngImagePlugin.PngImageFile image mode=L size=512x384 at 0x7AD3919CEF30>


In [38]:
import numpy as np

image = np.array(train[0]["image"])
mask = np.array(train[0]["label"])

print("Image shape:", image.shape, image.dtype)
print("Mask shape:", mask.shape, mask.dtype)
print("Unique mask values:", np.unique(mask))


Image shape: (384, 512, 3) uint8
Mask shape: (384, 512) uint8
Unique mask values: [ 0 48 66 90]


In [39]:
small_train = ds["train"].select(range(1000))
small_val   = ds["validation"].select(range(200))

small_train.set_format("numpy", columns=["image", "label"])
small_val.set_format("numpy", columns=["image", "label"])



In [40]:
import tensorflow as tf

IMG_SIZE = 224

def preprocess(image, mask):
    image = tf.cast(image, tf.float32) / 255.0
    mask = tf.cast(mask, tf.int32)
    mask = mask[..., tf.newaxis]

    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    mask  = tf.image.resize(mask, (IMG_SIZE, IMG_SIZE), method="nearest")
    print("Image shape:", image.shape, image.dtype)
    print("Mask shape:", mask.shape, mask.dtype)

    return image, mask


In [41]:
img, msk = preprocess(ds["train"][0]["image"], ds["train"][0]["label"])

print("Image:", img.shape, img.dtype, tf.reduce_min(img).numpy(), tf.reduce_max(img).numpy())
print("Mask:", msk.shape, msk.dtype, tf.unique(tf.reshape(msk, [-1]))[0].numpy())



Image shape: (224, 224, 3) <dtype: 'float32'>
Mask shape: (224, 224, 1) <dtype: 'int32'>
Image: (224, 224, 3) <dtype: 'float32'> 0.0 1.0
Mask: (224, 224, 1) <dtype: 'int32'> [90  0 66 48]


In [48]:
BATCH_SIZE = 8

train_ds = train.to_tf_dataset(
    columns="image",
    label_cols="label",
    shuffle=True,
    batch_size=None
)

val_ds = validation.to_tf_dataset(
    columns="image",
    label_cols="label",
    shuffle=False,
    batch_size=None
)

train_ds = train_ds.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


Image shape: (224, 224, 3) <dtype: 'float32'>
Mask shape: (224, 224, 1) <dtype: 'int32'>
Image shape: (224, 224, 3) <dtype: 'float32'>
Mask shape: (224, 224, 1) <dtype: 'int32'>


In [49]:
import tensorflow as tf
import matplotlib.pyplot as plt

heights = []
widths = []
mask_heights = []
mask_widths = []

for i in range(100):   # just first 100 samples to test quickly
    img = ds["train"][i]["image"]
    msk = ds["train"][i]["label"]

    heights.append(img.size[1])   # PIL: size = (width, height)
    widths.append(img.size[0])
    mask_heights.append(msk.size[1])
    mask_widths.append(msk.size[0])

print("Unique image heights:", set(heights))
print("Unique image widths:", set(widths))
print("Unique mask heights:", set(mask_heights))
print("Unique mask widths:", set(mask_widths))


Unique image heights: {384, 2048, 256, 386, 512, 2560, 518, 1920, 392, 1537, 396, 2448, 273, 2706, 1427, 1936, 405, 1552, 534, 413, 1054, 800, 683, 428, 429, 1840, 816, 307, 435, 1664, 317, 3264, 1358, 338, 340, 341, 473, 218, 352, 356, 750, 366, 369, 250, 382, 383}
Unique image widths: {512, 1536, 1664, 256, 1024, 384, 642, 1920, 2048, 1280, 394, 534, 410, 2592, 550, 3264, 1600, 2114, 720, 2912, 612, 1000, 360}
Unique mask heights: {384, 2048, 256, 386, 512, 2560, 518, 1920, 392, 1537, 396, 2448, 273, 2706, 1427, 1936, 405, 1552, 534, 413, 1054, 800, 683, 428, 429, 1840, 816, 307, 435, 1664, 317, 3264, 1358, 338, 340, 341, 473, 218, 352, 356, 750, 366, 369, 250, 382, 383}
Unique mask widths: {512, 1536, 1664, 256, 1024, 384, 642, 1920, 2048, 1280, 394, 534, 410, 2592, 550, 3264, 1600, 2114, 720, 2912, 612, 1000, 360}
